In [ ]:
# install dependencies

In [1]:
# load dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dask_cudf as dd
import cudf

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# load and see train dataset
dataset = dd.read_csv("/kaggle/input/age-assessment-and-disease-risk-prediction/ai4bio_trainset/traindata.csv").head(10)
dataset.head()

,cpgsite,train10001,train10002,train10003,train10004,train10005,train10006,train10007,train10008,train10009,...,train18224,train18225,train18226,train18227,train18228,train18229,train18230,train18231,train18232,train18233
0,cg00050873,<NA>,<NA>,<NA>,<NA>,1.136022464,1.423834,<NA>,1.475799045,<NA>,...,<NA>,<NA>,<NA>,<NA>,2.079779018,1.991710808,2.020410174,1.814692208,1.550147352,2.142023166
1,cg00212031,<NA>,<NA>,<NA>,<NA>,-4.48985033,-9.210440,<NA>,-3.993781198,<NA>,...,<NA>,<NA>,<NA>,<NA>,-6.811544694,-3.886934792,-3.938986056,-3.343969964,-2.963760175,-2.881227756
2,cg00213748,<NA>,<NA>,<NA>,<NA>,<NA>,1.035353,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,cg00214611,<NA>,<NA>,<NA>,<NA>,-3.993781198,-3.837361,<NA>,-4.051632381,<NA>,...,<NA>,<NA>,<NA>,<NA>,-3.7013532,-3.256540279,-3.543689103,-3.580953124,-3.744756383,-3.7013532
4,cg00455876,<NA>,<NA>,<NA>,<NA>,1.312567124,1.288795,<NA>,1.475799045,<NA>,...,<NA>,<NA>,<NA>,<NA>,1.004557071,0.851872565,1.259524319,<NA>,1.443125668,<NA>


In [3]:
categorical_column = 'cpgsite'  
numerical_columns = dataset.columns.difference([categorical_column])

df_numerical = dataset[numerical_columns].astype('float64')
df_filled_numerical = df_numerical.fillna(0)
column_sums = df_filled_numerical.sum()
df_from_series = dd.from_cudf(
    cudf.DataFrame({'sample_id': column_sums.index, 'cpgsite_sum': column_sums.values}),
    npartitions=1
)
df_from_series.head()

,sample_id,cpgsite_sum
0,train10001,0.000000
1,train10002,0.000000
2,train10003,0.000000
3,train10004,-1.398461
4,train10005,-12.134430


In [ ]:
# # Convert dask_cudf DataFrame to Pandas DataFrame
# dataset_pd = dataset.compute().to_pandas()

# # Transpose the Pandas DataFrame
# transposed_dataset_pd = dataset_pd.T

# # Convert transposed Pandas DataFrame back to dask_cudf DataFrame
# transposed_dask_cudf = dask_cudf.from_cudf(pd.DataFrame(transposed_dataset_pd), npartitions=dataset.npartitions)

# # Display the transposed dask_cudf DataFrame
# transposed_dask_cudf.head()

In [5]:
dataset_map = dd.read_csv("/kaggle/input/age-assessment-and-disease-risk-prediction/ai4bio_trainset/trainmap.csv")
dataset_map['age'] = dataset_map['age'].astype(int)
dataset_map = dataset_map.iloc[:, 1:]
dataset_map.head()

,sample_id,age,gender,sample_type,disease
0,train10001,88,F,disease tissue,Alzheimer's disease
1,train10002,92,F,disease tissue,Alzheimer's disease
2,train10003,93,F,disease tissue,Alzheimer's disease
3,train10004,96,F,disease tissue,Alzheimer's disease
4,train10005,91,M,disease tissue,Alzheimer's disease


In [6]:
# map two dataframes
# merged_df = dask_cudf.merge(df1, df2, on='common_column')
merged_dataset = df_from_series.merge(dataset_map, on='sample_id', how='left')
merged_dataset = merged_dataset.sort_values('sample_id')
merged_dataset.head()

,sample_id,cpgsite_sum,age,gender,sample_type,disease
2032,train10001,0.000000,88,F,disease tissue,Alzheimer's disease
2033,train10002,0.000000,92,F,disease tissue,Alzheimer's disease
2034,train10003,0.000000,93,F,disease tissue,Alzheimer's disease
2035,train10004,-1.398461,96,F,disease tissue,Alzheimer's disease
2036,train10005,-12.134430,91,M,disease tissue,Alzheimer's disease


In [7]:
# save to final_train_data.csv
merged_dataset = merged_dataset.compute()
merged_dataset.to_csv("final_train_data.csv", index = False)

In [ ]:
# restart session with CPU 

In [8]:
dataset = pd.read_csv("/kaggle/working/final_train_data.csv")
dataset.head()

,sample_id,cpgsite_sum,age,gender,sample_type,disease
0,train10001,0.000000,88,F,disease tissue,Alzheimer's disease
1,train10002,0.000000,92,F,disease tissue,Alzheimer's disease
2,train10003,0.000000,93,F,disease tissue,Alzheimer's disease
3,train10004,-1.398461,96,F,disease tissue,Alzheimer's disease
4,train10005,-12.134430,91,M,disease tissue,Alzheimer's disease


In [12]:
# data preprocessing
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
dataset['gender'] = encoder.fit_transform(dataset['gender'])
dataset['sample_type'] = encoder.fit_transform(dataset['sample_type'])
dataset['disease'] = encoder.fit_transform(dataset['disease'])
dataset.head()

,sample_id,cpgsite_sum,age,gender,sample_type,disease
0,train10001,0.000000,88,0,1,0
1,train10002,0.000000,92,0,1,0
2,train10003,0.000000,93,0,1,0
3,train10004,-1.398461,96,0,1,0
4,train10005,-12.134430,91,1,1,0


In [16]:
from sklearn.model_selection import train_test_split

X,y = dataset[['cpgsite_sum', 'age', 'gender', 'sample_type']], dataset[['disease']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [30]:
# decision tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)

train_accuracy = accuracy_score(y_train, classifier.predict(X_train))
test_accuracy = accuracy_score(y_test, classifier.predict(X_test))

print("Train Accuracy : {}".format(train_accuracy))
print("Test Accuracy : {}".format(test_accuracy))

Train Accuracy : 0.9685696932887944
Test Accuracy : 0.8785670916818458


In [ ]:
# We can try any other models or automl

In [ ]:
# end